In [1]:
from helpers.config import vectorstore
vectorstore.get()

c:\PythonWorkspace\server-pdf_handle\Server-RAG\helpers\config.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")
c:\PythonWorkspace\server-pdf_handle\Server-RAG\helpers\config.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'ids': ['bf89eaa1-2f9e-4d2d-9344-119021a7e6be',
  '62ff3a37-7ee2-4a7d-ae52-49a1567a67a7',
  '07b27390-2da7-4a3d-a2c5-09c940e549d0',
  '077f3a75-60d6-4e89-8b24-96f01d47ff93',
  '0b5ccb7f-d62f-4e04-ad8a-93544cca1449',
  '956f77df-5a3e-472f-ba21-c4b1c78a5598',
  '268ea363-062d-47c6-b7b3-43ffabfc4f70',
  '442fcadc-acf8-4edd-bc9f-1d729dbec0d9',
  'e3cb4731-80cd-440f-a463-56c2ce24d76c',
  '3fee9cde-2ae1-4a1a-a26d-f8cdb6ffa31c',
  'a906a909-f20c-4918-b651-dee845192a18',
  '96ba5bb1-804a-4dd5-8e17-7ddfc911d773',
  '7f47f2bd-f685-4b88-8d17-680703a7d9d0',
  '006fc0a0-5b78-4546-be2e-27a7bce12e01',
  'd2cfffb7-b8b2-4d05-babf-e4a739d13b37',
  '555ca38f-69a5-4813-a430-7adf28db63fb',
  '5cecb4eb-888c-416a-b443-2bb63a6b7c0d',
  '90942836-1983-4c07-8575-f16632534d67',
  'f3d41f8a-ba23-4d86-9fae-b50e51e3babc',
  'db1a9735-753d-46e5-a827-03eb43868eff',
  'a8a91940-8706-4baf-b130-fc36c3c3f54b',
  '9eabe17f-8f2a-4e0f-bf21-1539033a2ebc',
  '309e7cdf-b99e-4b28-b90c-d9c614dcfd66',
  'dcfcaee3-4bc9-445d-aab7-

In [3]:
print(vectorstore.get().keys()) 

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])


In [6]:
print(vectorstore.get()["metadatas"])

[{'author': 'MCC Support', 'creationDate': "D:20241228184304+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 5, 'format': 'PDF 1.5', 'heading': 'Foreword', 'keywords': '<keyword[, keyword, ]>', 'modDate': 'D:20241228184304Z', 'producer': 'www.ilovepdf.com', 'source': '36201-i00.pdf', 'start_page': 4, 'subject': '<Title 1; Title 2> (Release 14 | 13 |12)', 'title': '3GPP TS ab.cde'}, {'author': 'MCC Support', 'creationDate': "D:20241226220500+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 10, 'format': 'PDF 1.5', 'heading': '4.1 Overview', 'modDate': 'D:20241226220500Z', 'producer': 'www.ilovepdf.com', 'source': '38304-gb0.pdf', 'start_page': 9, 'subject': 'NR; User Equipment (UE) procedures in Idle mode and RRC Inactive state (Release 16)', 'title': '3GPP TS 38.304'}, {'author': 'MCC Support', 'creationDate': "D:20241226220500+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 10, 'format': 'PDF 1.5', 'heading': '4.1 Overview', 'modDate': 'D:20241226220500Z', 'prod

In [12]:
print(vectorstore.get(where={"source": "36201-i00.pdf"})["ids"])

['bf89eaa1-2f9e-4d2d-9344-119021a7e6be']


In [ ]:
import pandas as pd
from IPython.display import display

# ChromaDB'den verileri al
vector_data = vectorstore.get()

# None olan anahtarları kontrol et
keys = ['ids', 'documents', 'metadatas']
valid_keys = [key for key in keys if vector_data.get(key) is not None]

# En küçük uzunluğu belirle (tüm sütunların eşit uzunlukta olması için)
min_length = min(len(vector_data[key]) for key in valid_keys)

# DataFrame oluştur
df = pd.DataFrame({
    'id': vector_data['ids'][:min_length],
    'document': vector_data['documents'][:min_length],
    'metadata': vector_data['metadatas'][:min_length]
})

# Tüm satır ve sütunların görüntülenmesini sağla
pd.set_option('display.max_rows', None)  # Tüm satırları göster
pd.set_option('display.max_columns', None)  # Tüm sütunları göster
pd.set_option('display.width', None)  # Çerçeveyi genişlet
pd.set_option('display.max_colwidth', None)  # Tüm hücrelerin içeriğini göster

display(df)


,id,document,metadata
0,bf89eaa1-2f9e-4d2d-9344-119021a7e6be,3GPP \n3GPP TS 36.201 V18.0.0 (2024-03) \n5 \n...,"{'author': 'MCC Support', 'creationDate': 'D:2..."
1,62ff3a37-7ee2-4a7d-ae52-49a1567a67a7,3GPP \n3GPP TS 38.304 V16.11.0 (2024-12) \n10\...,"{'author': 'MCC Support', 'creationDate': 'D:2..."
2,07b27390-2da7-4a3d-a2c5-09c940e549d0,expires while MICO mode is activated it is up ...,"{'author': 'MCC Support', 'creationDate': 'D:2..."
3,077f3a75-60d6-4e89-8b24-96f01d47ff93,3GPP \n3GPP TS 38.304 V16.11.0 (2024-12) \n12\...,"{'author': 'MCC Support', 'creationDate': 'D:2..."
4,0b5ccb7f-d62f-4e04-ad8a-93544cca1449,"For a UE operating in SNPN access mode, \nperf...","{'author': 'MCC Support', 'creationDate': 'D:2..."
...,...,...,...
1411,7b87a5c0-01e1-4833-83a5-9402f329b432,"PMLR, 2017, pp.\n1126–1135. [Online]. Availabl...","{'creationDate': 'D:20250120133513+04'00'', 'c..."
1412,7f309914-f977-4478-b61a-cb5a45d8cce8,και\nυλoπo´ιηση\nαλγoρ´ιθµων\nτεχνητ ´ης\nνoηµ...,"{'creationDate': 'D:20250120133513+04'00'', 'c..."
1413,4096ed51-cdbe-4589-9618-1bb82be640a0,"slicing for virtualized beyond 5g networks,” C...","{'creationDate': 'D:20250120133513+04'00'', 'c..."
1414,200f43c9-00ce-46bb-a933-c4baeb54ad34,"data for large language models,” Journal of th...","{'creationDate': 'D:20250120133513+04'00'', 'c..."
